In [18]:
import numpy as np 
import matplotlib.pyplot as plt 

In [19]:
def phi_calc(Xi,Xj,N_internal, Xj_neighbours):
    Ni = Xi.shape[0]
    Nj = Xj.shape[0]
    phi_mat = np.zeros((Ni,Nj))
    dphidx_mat = np.zeros((Ni,Nj))
    dphidz_mat = np.zeros((Ni,Nj))
    nablaphi_mat = np.zeros((Ni,Nj))


    #taking the ith point from Xi:
    for i in range(0,Ni):

        x = Xi[i,0]
        z = Xi[i,1]

        #finding the nearest point in the internal points in Xj
        nearest_point = np.argmin((Xj[0:N_internal,0]-x)**2 + (Xj[0:N_internal,1]-z)**2)
        #print(nearest_point)

        #the indices of points used for interpolation is therefore given by the neighbours of the nearest point
        #Xj_neighbours[nearest_point,:]

        #which have a coordinate set:
        X = Xj[Xj_neighbours[nearest_point,:],:]

        #fitting polynomial:
        poly_mat = np.zeros((9,9))

        for j in range(0,9):
            poly_mat[j,0] = 1
            poly_mat[j,1] = X[j,0]
            poly_mat[j,2] = X[j,1]
            poly_mat[j,3] = X[j,0]**2
            poly_mat[j,4] = X[j,1]**2
            poly_mat[j,5] = X[j,0]*X[j,1]
            poly_mat[j,6] = X[j,0]**2*X[j,1]
            poly_mat[j,7] = X[j,1]**2*X[j,0]
            poly_mat[j,8] = X[j,0]**2*X[j,1]**2


        #matrix of polynomial coefficients
        #print(i)
        
        coef_mat = np.linalg.inv(poly_mat)


        #calculating the value of the 9 polynomials at point (x,z)
        polynomial = np.zeros((1,9))
        for j in range(0,9):
            polynomial[:,j] = coef_mat[0,j] + coef_mat[1,j]*x + coef_mat[2,j]*z + coef_mat[3,j]*x*x + coef_mat[4,j]*z*z 
            polynomial[:,j] += coef_mat[5,j]*x*z + coef_mat[6,j]*x*x*z + coef_mat[7,j]*x*z*z + coef_mat[8,j]*x*x*z*z

        #calculating value of derivatives at point (x,z)
        dpdx = np.zeros((1,9))
        for j in range(0,9):
            dpdx[:,j] = coef_mat[1,j] + 2*coef_mat[3,j]*x + coef_mat[5,j]*z + coef_mat[6,j]*x*z*2 + coef_mat[7,j]*z*z + coef_mat[8,j]*x*z*z*2

        dpdz = np.zeros((1,9))
        for j in range(0,9):
            dpdz[:,j] = coef_mat[2,j] + 2*coef_mat[4,j]*z + coef_mat[5,j]*x + coef_mat[6,j]*x*x + coef_mat[7,j]*x*z*2 + coef_mat[8,j]*x*x*z*2

        d2pdx2 = np.zeros((1,9))
        for j in range(0,9):
            d2pdx2[:,j] = 2*coef_mat[3,j] + coef_mat[6,j]

        d2pdz2 = np.zeros((1,9))
        for j in range(0,5):
            d2pdx2[:,j] = 2*coef_mat[4,j]



        nabla = np.zeros((1,9))
        for j in range(0,5):
            nabla[:,j] = 2*coef_mat[3,j] + 2*coef_mat[4,j]

        #inserting vector of polynomial values back into the phi matrix:
        phi_mat[i,Xj_neighbours[nearest_point,:]] = polynomial
        dphidx_mat[i,Xj_neighbours[nearest_point,:]] = dpdx
        dphidz_mat[i,Xj_neighbours[nearest_point,:]] = dpdz
        nablaphi_mat[i,Xj_neighbours[nearest_point,:]] = nabla

    return phi_mat, dphidx_mat, dphidz_mat, nablaphi_mat

In [20]:
#generating grid

dx = 0.1
xx,zz = np.meshgrid(np.arange(0,1+dx,dx),np.arange(0,1+dx,dx))
#splitting into internal and external points:
xxi = xx[1:-1,1:-1]
zzi = zz[1:-1,1:-1]

nzi= xxi.shape[0]
nxi = xxi.shape[1]


Xji = np.stack([xxi.flatten(),zzi.flatten()],axis=1) 

Tpoint = 5
Xbc1 = np.stack([xx[:,0:1].flatten(),zz[:,0:1].flatten()],axis=1) 
Xbc2 = np.stack([xx[:,-1:].flatten(),zz[:,-1:].flatten()],axis=1) 
Xbc3 = np.stack([xx[-1:,1:-1].flatten(),zz[-1,1:-1].flatten()],axis=1) 
Xbc4 = np.stack([xx[0:1,1:Tpoint].flatten(),zz[0:1,1:Tpoint].flatten()],axis=1) 
Xbc5 = np.stack([xx[0:1,Tpoint:-1].flatten(),zz[0:1,Tpoint:-1].flatten()],axis=1) 


N_internal = Xji.shape[0]

Xj = np.concatenate([Xji,Xbc1,Xbc2,Xbc3,Xbc4,Xbc5])
Nj = Xj.shape[0]
bc1 = N_internal + Xbc1.shape[0]
bc2 = bc1 + Xbc2.shape[0]
bc3 = bc2 + Xbc3.shape[0]
bc4 = bc3 + Xbc4.shape[0]
bc5 = bc4 + Xbc5.shape[0]

In [21]:
#we now have a second vector of points that we wish to perform an interpolation at:

Xi = Xj + 0

#finding nnearest neighbours of each point in Xj
r2 = (Xj[:,0:1]-Xj[:,0:1].transpose())**2 + (Xj[:,1:2]-Xj[:,1:2].transpose())**2

Xj_neighbours = np.zeros((Xj.shape[0],9),dtype = int)

Xj_neighbours = np.zeros((Xj.shape[0],9),dtype = int)
for i in range(0,Xj.shape[0]):
    Xj_neighbours[i,:] = (np.argpartition(r2[:,i],9)[0:9])


phi_mat, dphidx_mat, dphidz_mat, nablaphi_mat = phi_calc(Xi,Xj,N_internal, Xj_neighbours)